In [2]:
import requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
import pandas as pd
import requests
import re

In [3]:
def get_sgp_office_data(start, end):
    sgp_office_data = []
    for offset in range(start, end):
        url = f'https://www.99.co/singapore/commercial/rent/offices?listing_type=rent&main_category=office&map_bounds=1.5827095153768858%2C103.49449749970108%2C1.1090706240313446%2C104.12483807587296&page_num={offset}&page_size=35&path=%2Fsingapore%2Fcommercial%2Frent%2Foffices&property_segments=commercial&query_coords=1.3039947%2C103.8298507&query_limit=radius&query_type=city&radius_max=1000&rental_type=unit&show_cluster_preview=true&show_description=true&show_internal_linking=true&show_meta_description=true&show_nearby=true&zoom=15'
        html = requests.get(url, headers={'User-Agent': "Mozilla/5.0"})
        soup = BeautifulSoup(html.content, "html.parser")
        
        tags_name = soup.find_all("a", class_="_3Ajbv _30I97 _1vzK2", href=True)
        tags_district = soup.find_all("li", class_="_3WG9R", href=False, attrs={"itemprop": "addressRegion"})
        tags_address = soup.find_all("li", class_="_3WG9R _3L5OV", attrs={"itemprop": "streetAddress"})
        tags_price = soup.find_all("li", class_="JlU_W")
        tags_size = soup.find_all("li", class_="_1x-U1")
        tags_link = soup.find_all("a", href=True)

        # Extract the text values from the tags
        names = [i.text.replace("Office in ", "").strip() for i in tags_name]
        districts = [i.text.strip() for i in tags_district]
        addresses = [i.text.strip() for i in tags_address]
        prices = [i.text.strip().replace(",", "").replace("[", "").replace("]", "").replace("$", "").split("/")[0] for i in tags_price]
        sizes = [i.text.strip().replace(",", "").replace("[", "").replace("]", "").split("/")[0].split(" ")[0] for i in tags_size]
        links = [i['href'] for i in tags_link if 'enquiry_source' in i['href']]
        # Zip the values together
        rows = list(zip(names, districts, addresses, prices, sizes, links))
        sgp_office_data.extend(rows)
        
    df_sgp_office = pd.DataFrame(sgp_office_data, columns=['Name', 'District', 'Address', 'Price ($) p/m', 'Size (m2)', 'Link'])
    df_sgp_office['Price per m2'] =  (df_sgp_office['Price ($) p/m'].astype(float) / df_sgp_office['Size (m2)'].astype(float)).round(2)
    df_sgp_office['Link'] = df_sgp_office['Link'].apply(lambda x: 'https://www.99.co' + x)
    df_sgp_office = df_sgp_office.reindex(columns=[i for i in df_sgp_office.columns if i != 'Link'] + ['Link'])

    return df_sgp_office


In [4]:
df_sgp_office = get_sgp_office_data(1, 30)

In [5]:
df_sgp_office

,Name,District,Address,Price ($) p/m,Size (m2),Price per m2,Link
0,New Bridge Road,D1,New Bridge Rd,12000,1373,8.74,https://www.99.co/singapore/commercial/rent/pr...
1,Parkway Parade,D15,Marine Parade Rd,6200,956,6.49,https://www.99.co/singapore/commercial/rent/pr...
2,Centennial Tower,D1,Temasek Ave,31902,2454,13.00,https://www.99.co/singapore/commercial/rent/pr...
3,Amoy Street,D1,Amoy St,4600,894,5.15,https://www.99.co/singapore/commercial/rent/pr...
4,Regency House,D9,Penang Rd,18241,2465,7.40,https://www.99.co/singapore/commercial/rent/pr...
...,...,...,...,...,...,...,...
928,Chinatown Point,D1,New Bridge Rd,36650,4887,7.50,https://www.99.co/singapore/commercial/rent/pr...
929,ESR BizPark @ Chai Chee,D16,Chai Chee Rd,39485,10968,3.60,https://www.99.co/singapore/commercial/rent/pr...
930,GB Building,D1,Cecil St,32550,5425,6.00,https://www.99.co/singapore/commercial/rent/pr...
931,Fusionopolis Place,D5,Fusionopolis Pl,8681,1467,5.92,https://www.99.co/singapore/commercial/rent/pr...


In [6]:
df_sgp_office = df_sgp_office.drop(index=244)
df_sgp_office = df_sgp_office.drop(index=38)

In [7]:
df_sgp_office.to_csv(r'./data/office_sgp.csv')

NameError: name 'df_district_avg' is not defined

In [8]:
df_sgp_office[df_sgp_office['District'] == 'D7']

,Name,District,Address,Price ($) p/m,Size (m2),Price per m2,Link
14,Sunshine Plaza,D7,Bencoolen St,4000,517,7.74,https://www.99.co/singapore/commercial/rent/pr...
18,GR.ID,D7,Selegie Rd,128000,15000,8.53,https://www.99.co/singapore/commercial/rent/pr...
26,Icb Enterprise House,D7,Middle Rd,1200,250,4.80,https://www.99.co/singapore/commercial/rent/pr...
33,Jun Xin Building,D7,Nth Bridge Rd,15870,3170,5.01,https://www.99.co/singapore/commercial/rent/pr...
46,Fortune Centre,D7,Middle Rd,8000,1668,4.80,https://www.99.co/singapore/commercial/rent/pr...
...,...,...,...,...,...,...,...
886,Golden Mile Tower,D7,Beach Rd,2200,625,3.52,https://www.99.co/singapore/commercial/rent/pr...
887,Golden Mile Tower,D7,Beach Rd,2200,625,3.52,https://www.99.co/singapore/commercial/rent/pr...
888,Golden Mile Tower,D7,Beach Rd,2200,625,3.52,https://www.99.co/singapore/commercial/rent/pr...
916,The Plaza,D7,Beach Rd,18000,3000,6.00,https://www.99.co/singapore/commercial/rent/pr...
